In [6]:
import json
from dotenv import load_dotenv
import os
from pydantic import BaseModel
from openai import OpenAI
from datetime import datetime
from typing import List
from fastapi.encoders import jsonable_encoder
import pandas as pd



# Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?
# 11 25

# client = OpenAI()
# client.api_key = os.getenv('OPENAI_API_KEY')

class ReceiptDetail(BaseModel):
    quantity: float
    description: str
    category: str
    unit_price: float
    subtotal: float

class PaymentMethod(BaseModel):
    payment_method_type: str
    payment_method_name: str
    paid_amount: float

class Transaction(BaseModel):
    transaction_date: str
    commerce: str
    receipt_detail: List[ReceiptDetail]
    receipt_payment_method: List[PaymentMethod]
    receipt_total: float
    total_payment: float

Transaction.model_rebuild() # This is required to enable recursive types

class Response(BaseModel):
    transaction: Transaction

user_input_lang = 'spanish'

# aqui tiene que ir el correspondiente archivo de la carpeta landing
user_input_text = """
acabamos de comprar en costco un par de jeans por 500 unos cuernitos por 99 dos pastelitos por 150 cada uno 25 litros de gasolina por 23.01 cada uno 1.30 kilos de tomate en 29.90 cada uno pagado con tarjeta de crédito costco citibanamex un total de 1500.12 pesos y con tarjeta NU un total de 2000.01 pesos
"""

EXECUTION_DATETIME = datetime.now().strftime('%Y%m%d_%H%M%S') # será compartido en toda la ejecución para referenciar el mismo archivo en todas las capas

load_dotenv(dotenv_path='./.env')

client = OpenAI(
    base_url="https://api.moonshot.ai/v1",
    api_key=os.getenv('MOONSHOT_API_KEY'),
)

today_date = datetime.today().strftime('%Y-%m-%d')

chat_completion = client.chat.completions.create(
        model="kimi-k2.5",
        # messages=[{"role": "user", "content": "Hello"}],
        messages=[
        {"role": "system", "content": f"""
You are an expert at structured data extraction. You will be given unstructured text with detailed receipts from purchases in commerces and detailed payment methods of that specific receipt and should convert it into the given structure. Consider todays date as {today_date} and consider you will receive the data in {user_input_lang} but the structure is in english.
        """ },
        {"role": "user", "content": user_input_text}
    ],
)

# print(chat_completion.choices[0].message.content)
# print(chat_completion.usage.prompt_tokens, chat_completion.usage.completion_tokens)



response = chat_completion.choices[0].message.content ### landing

print(response)
# Transform response in py dict
# response_json = jsonable_encoder(response)['transaction']
# print(response_json)

```json
{
  "receipt": {
    "merchant": "Costco",
    "date": "2026-02-03",
    "currency": "MXN",
    "items": [
      {
        "description": "Jeans",
        "quantity": 1,
        "unit": "pair",
        "unit_price": 500.0,
        "total_price": 500.0
      },
      {
        "description": "Cuernitos",
        "quantity": 1,
        "unit": "package",
        "unit_price": 99.0,
        "total_price": 99.0
      },
      {
        "description": "Pastelitos",
        "quantity": 2,
        "unit": "piece",
        "unit_price": 150.0,
        "total_price": 300.0
      },
      {
        "description": "Gasolina",
        "quantity": 25,
        "unit": "liter",
        "unit_price": 23.01,
        "total_price": 575.25
      },
      {
        "description": "Tomate",
        "quantity": 1.3,
        "unit": "kg",
        "unit_price": 29.9,
        "total_price": 38.87
      }
    ],
    "payment_methods": [
      {
        "type": "credit_card",
        "provider": "Citiban

In [7]:
print(chat_completion)

ChatCompletion(id='chatcmpl-6982da597bf13000c9d9b828', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "receipt": {\n    "merchant": "Costco",\n    "date": "2026-02-03",\n    "currency": "MXN",\n    "items": [\n      {\n        "description": "Jeans",\n        "quantity": 1,\n        "unit": "pair",\n        "unit_price": 500.0,\n        "total_price": 500.0\n      },\n      {\n        "description": "Cuernitos",\n        "quantity": 1,\n        "unit": "package",\n        "unit_price": 99.0,\n        "total_price": 99.0\n      },\n      {\n        "description": "Pastelitos",\n        "quantity": 2,\n        "unit": "piece",\n        "unit_price": 150.0,\n        "total_price": 300.0\n      },\n      {\n        "description": "Gasolina",\n        "quantity": 25,\n        "unit": "liter",\n        "unit_price": 23.01,\n        "total_price": 575.25\n      },\n      {\n        "description": "Tomate",\n        "quantity"

In [ ]:
# Save into to_text layer
today_date_folder = datetime.today().strftime('%Y%m%d')
os.makedirs(f'./spend/text_to_json/{today_date_folder}',exist_ok=True)

In [ ]:
filename = f'receipt_payments_{EXECUTION_DATETIME}.json'
filepath_json = f'./spend/text_to_json/{today_date_folder}/{filename}'

with open(filepath_json, 'w') as f:
    json.dump(response_json, f)

NameError: name 'EXECUTION_DATETIME' is not defined

In [ ]:
%store filename
%store filepath_json
%store today_date_folder
%store EXECUTION_DATETIME

UsageError: Unknown variable 'filename'
